# Web crawler work

Because NTU course page was coded by mixed and complicated way which made it hard to crawl by only using resquest or BeautifulSoup package. So I combine BeautifulSoup and Selenium (other library) to do this work.

Selenium<br>
Reference: http://selenium-python.readthedocs.io/ <br>
           http://selenium-python-zh.readthedocs.io/en/latest/installation.html (Chinese)
install: pip install selenium

BeautifulSoup <br>
Reference: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd #use to form dataframe

In [2]:
URI='https://nol.ntu.edu.tw/nol/coursesearch/search_for_02_dpt.php?current_sem=106-2&coursename=&dptname=2270&dpt_sel=2000&yearcode=0&teachername=&selcode=-1&alltime=yes&week1=&week2=&week3=&week4=&week5=&week6=&allproced=yes&proced0=&proced1=&proced2=&proced3=&proced4=&procedE=&proced5=&proced6=&proced7=&proced8=&proced9=&procedA=&procedB=&procedC=&procedD=&allsel=yes&selCode1=&selCode2=&selCode3=&page_cnt=15'
driver=webdriver.Chrome()
driver.get(URI)
soup = BeautifulSoup(driver.page_source, "lxml")
jump = soup.find_all('select',{'name':'jump'})
page_numb = len(jump[0].find_all('option'))
option = 1
next_page = '/html/body/table[2]/tbody/tr/td/font/table[5]/tbody/tr[2]/td[2]/select/option['+str(option)+']'
all_page = list()
while option <= page_numb:
    driver.find_element_by_xpath(next_page).click()
    option+= 1
    next_page = '/html/body/table[2]/tbody/tr/td/font/table[5]/tbody/tr[2]/td[2]/select/option['+str(option)+']'
    soup = BeautifulSoup(driver.page_source, "lxml")
    all_page.append(soup)
    
    


Create empty dataframe for storing data and use for loop to fill it.

In [3]:
df = pd.DataFrame(columns=range(0,17))
for page in all_page:
    table = page.find_all('table')
    table = table[7].find('tbody').find_all('tr')
    row_number = len(table)
    new_table = pd.DataFrame(columns=range(0,17), index = [i for i in range((row_number-1))]) # I know the size 
    for i in range(1,row_number):
        columns = table[i].find_all('td')
        column_marker = 0
        for column in columns:
            new_table.iat[i-1,column_marker] = column.get_text()
            column_marker += 1
    df = pd.concat([df,new_table],axis=0, ignore_index=True)
    
col_name = [tag.get_text() for tag in table[0].find_all('td')] 
df.columns = col_name

To check if dataframe store repetive data. If it did, there must be something wrong.

In [4]:
len(df['流水號']) == len(df['流水號'].unique())

True

In [5]:
df.head()

,流水號,授課對象,課號,班次,課程名稱查看課程大綱，請點選課程名稱,學分,課程識別碼,全/半年,必/選修,授課教師,加選方式,時間教室,總人數,選課限制條件,備註,課程網頁,本學期我預計要選的課程
0,63941,心理所,Psy8999,,博士論文,0.0,227 D0010,半年,必修,,1,,30,本校修課人數上限：30人,,,加入
1,34110,心理所,Psy8001,,專題討論,1.0,227 D0060,半年,必修,黃囇莉,2,三7(北館視聽),60,本校修課人數上限：60人,博班請選此課.2:20開始上課.,,加入
2,87859,心理所,Psy8006,,臨床神經心理學特論一,2.0,227 D0130,半年,選修,張玉玲,2,"二2,3(南館S217)",15,本校修課人數上限：15人,,,加入
3,57260,心理所,Psy8161,,記憶特論二,3.0,227 D1040,半年,選修,葉怡玉,2,"三2,3,4(南館S309)",5,本校修課人數上限：5人,時地另訂,,加入
4,21792,心理所,Psy8160,,腦磁圖專題研究(二),2.0,227 D1080,半年,選修,郭柏呈,2,,6,"本課有先修科目規定,本校修課人數上限：6人",,,加入


In [6]:
#to save page
#df.to_csv('~.csv') 

# Next step?

1.This work is only get course information from the department of psychology. It can also download course from other department.<br>
2.This work use lots of xpath, it make this code very fragile. Is there any way to polish it?